### Import Hubway database

In [1]:
import sqlite3
import pandas as pd
db = sqlite3.connect('hubway.db')
def run_query(query):
    return pd.read_sql_query(query,db)

# Pull data from the trips table

### What was the duration of the longest trip?

In [2]:
query = '''
SELECT duration FROM trips
ORDER BY duration DESC
LIMIT 1;
'''
run_query(query)

,duration
0,9999


#### Return the 10 highest trip durations

In [3]:
query = '''
SELECT duration FROM trips
ORDER BY duration DESC
LIMIT 10;
'''
run_query(query)

,duration
0,9999
1,9998
2,9998
3,9997
4,9996
5,9996
6,9995
7,9995
8,9994
9,9994


#### Return every column in trips where the duration was longer than 9990 seconds

In [4]:
query = '''
SELECT * FROM trips
WHERE duration > 9990;
'''
run_query(query)

,id,duration,start_date,start_station,end_date,end_station,bike_number,sub_type,zip_code,birth_date,gender
0,4768,9994,2011-08-03 17:16:00,22,2011-08-03 20:03:00,24,B00002,Casual,,,
1,8448,9991,2011-08-06 13:02:00,52,2011-08-06 15:48:00,24,B00174,Casual,,,
2,11341,9998,2011-08-09 10:42:00,40,2011-08-09 13:29:00,42,B00513,Casual,,,
3,24455,9995,2011-08-20 12:20:00,52,2011-08-20 15:07:00,17,B00552,Casual,,,
4,55771,9994,2011-09-14 15:44:00,40,2011-09-14 18:30:00,40,B00139,Casual,,,
5,81191,9993,2011-10-03 11:30:00,22,2011-10-03 14:16:00,36,B00474,Casual,,,
6,89335,9997,2011-10-09 02:30:00,60,2011-10-09 05:17:00,45,B00047,Casual,,,
7,124500,9992,2011-11-09 09:08:00,22,2011-11-09 11:55:00,40,B00387,Casual,,,
8,133967,9996,2011-11-19 13:48:00,4,2011-11-19 16:35:00,58,B00238,Casual,,,
9,147451,9996,2012-03-23 14:48:00,35,2012-03-23 17:35:00,33,B00550,Casual,,,


#### Return trips with a duration longer than 9990 by 'registered' users

In [5]:
query = '''
SELECT * FROM trips
WHERE (duration >= 9990) AND (sub_type = "Registered")
ORDER BY duration DESC;
'''
run_query(query)

,id,duration,start_date,start_station,end_date,end_station,bike_number,sub_type,zip_code,birth_date,gender
0,315737,9995,2012-07-03 18:28:00,12,2012-07-03 21:15:00,12,B00250,Registered,'02120,1964.0,Male


### How many trips were taken by 'registered' users?

In [6]:
query = '''
SELECT COUNT(*) AS "Total Trips by Registered Users"
FROM trips
WHERE sub_type = "Registered";
'''
run_query(query)

,Total Trips by Registered Users
0,1105192


### What was the average trip duration?

In [7]:
query = '''
SELECT AVG(duration) AS "Average Duration"
FROM trips;
'''
run_query(query)

,Average Duration
0,912.409682


### Do registered or casual users take longer trips?

In [8]:
query = '''
SELECT sub_type, AVG(duration) AS "Average Duration"
FROM trips
GROUP BY sub_type;
'''
run_query(query)

,sub_type,Average Duration
0,Casual,1519.643897
1,Registered,657.026067


### Which bike was used for the most trips?

In [9]:
query = '''
SELECT bike_number as "Bike Number", COUNT(*) AS "Number of Trips"
FROM trips
GROUP BY bike_number
ORDER BY COUNT(*) DESC
LIMIT 1;
'''
run_query(query)

,Bike Number,Number of Trips
0,B00490,2120


### What is the average duration of trips by users over the age of 30?

In [10]:
query = '''
SELECT AVG(duration) FROM trips
WHERE (2017 - birth_date) > 30;
'''
run_query(query)

,AVG(duration)
0,923.014685


# Include data from the stations table

In [11]:
query = '''
SELECT * FROM stations
LIMIT 5;
'''
run_query(query)

,id,station,municipality,lat,lng
0,3,Colleges of the Fenway,Boston,42.340021,-71.100812
1,4,Tremont St. at Berkeley St.,Boston,42.345392,-71.069616
2,5,Northeastern U / North Parking Lot,Boston,42.341814,-71.090179
3,6,Cambridge St. at Joy St.,Boston,42.361284999999995,-71.06514
4,7,Fan Pier,Boston,42.353412,-71.044624


### Which station is the most frequent starting point?

In [12]:
query = '''
SELECT stations.station AS "Station", COUNT(*) AS "Count"
FROM trips INNER JOIN stations
ON trips.start_station = stations.id GROUP BY stations.station ORDER BY COUNT(*) DESC
LIMIT 5;
'''
run_query(query)

,Station,Count
0,South Station - 700 Atlantic Ave.,56123
1,Boston Public Library - 700 Boylston St.,41994
2,Charles Circle - Charles St. at Cambridge St.,35984
3,Beacon St / Mass Ave,35275
4,MIT at Mass Ave / Amherst St,33644


### Which stations are most frequently used for round trips?

In [13]:
query = '''
SELECT stations.station AS "Station", COUNT(*) AS "Count"
FROM trips INNER JOIN stations
ON trips.start_station = stations.id
WHERE trips.start_station = trips.end_station
GROUP BY stations.station
ORDER BY COUNT(*) DESC
LIMIT 5;
'''
run_query(query)

,Station,Count
0,The Esplanade - Beacon St. at Arlington St.,3064
1,Charles Circle - Charles St. at Cambridge St.,2739
2,Boston Public Library - 700 Boylston St.,2548
3,Boylston St. at Arlington St.,2163
4,Beacon St / Mass Ave,2144


### How many trips start and end in different municipalities?

In [14]:
query = '''
SELECT COUNT(trips.id) AS "Count"
FROM trips INNER JOIN stations AS start
ON trips.start_station = start.id
INNER JOIN stations AS end
ON trips.end_station = end.id
WHERE start.municipality <> end.municipality;
'''
run_query(query)

,Count
0,309748


### How many trips incurred additional fees (lasted longer than 30 minutes)?

In [24]:
query = '''
SELECT COUNT(duration) AS "Count"
FROM trips
WHERE duration > 1800;
'''
run_query(query)

,Count
0,123155


### Which bike was used for the longest total time?

In [63]:
query = '''
SELECT bike_number as "Bike Number", COUNT(duration) as "Total Duration"
FROM trips
GROUP BY bike_number
ORDER BY COUNT(duration) DESC
LIMIT 5;
'''
run_query(query)

,Bike Number,Total Duration
0,B00490,2120
1,B00268,2104
2,B00548,2099
3,B00559,2083
4,B00563,2067


### Did registered or casual users take more round trips?

In [62]:
query = '''
SELECT sub_type, COUNT(*) AS "Count"
FROM trips INNER JOIN stations
ON trips.start_station = stations.id
WHERE trips.start_station = trips.end_station
GROUP BY sub_type
'''
run_query(query)

,sub_type,Count
0,Casual,41427
1,Registered,31635


### Which municipality had the longest average duration?

In [76]:
query = '''
SELECT municipality, AVG(duration) avg_duration
FROM trips INNER JOIN stations
WHERE trips.start_station = stations.id
GROUP BY municipality
ORDER BY 2 DESC;
'''
run_query(query)

,municipality,avg_duration
0,Brookline,1020.844836
1,Boston,929.211749
2,Somerville,866.462890
3,Cambridge,842.171816
